<a href="https://colab.research.google.com/github/RYAN073-AI/-S-P-500-Stock-Ranking-System/blob/main/S%26P_500_Stock_Ranking_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df1=pd.read_csv("/content/sp500_companies.csv")#file path

In [ ]:
new_df=df1[['Symbol','Shortname']]

In [ ]:
new_df

Extracting Stock Data from Yfinance


In [ ]:
import numpy as np
import yfinance as yf

def get_stock_info(symbol):
    try:
        stock = yf.Ticker(symbol)  # creating a ticker object for the particular stock
        information = stock.info
        hist = stock.history(period="1y")  # history of the stock for 1 year

        # Calculate 1-year returns
        if len(hist) > 0:
            returns_1y = (hist['Close'].iloc[-1] - hist['Close'].iloc[0]) / hist['Close'].iloc[0]
        else:
            returns_1y = 0

        # Extract key metrics
        data = {
            'Symbol': symbol,
            'PE_Ratio': information.get('trailingPE', np.nan),
            'Price_to_Book': information.get('priceToBook', np.nan),
            'ROE': information.get('returnOnEquity', np.nan) * 100 if information.get('returnOnEquity') else np.nan,
            'Profit_Margin': information.get('profitMargins', np.nan) * 100 if information.get('profitMargins') else np.nan,
            'Debt_to_Equity': information.get('debtToEquity', np.nan),
            'Revenue_Growth': information.get('revenueGrowth', np.nan) * 100 if information.get('revenueGrowth') else np.nan,
            'Current_Ratio': information.get('currentRatio', np.nan),
            'Returns_1Y': returns_1y
        }

        return data

    except Exception as e:

        return None


In [ ]:
df1=pd.DataFrame()
data=[]
stock_list=new_df['Symbol'].tolist()
for i in range(400):
  stock_data=get_stock_info(stock_list[i])
  if(stock_data is not None):
    data.append(stock_data)
df1=pd.DataFrame(data)




In [ ]:
df1.head()

Create Stock Rating


In [ ]:
def create_stock_rating(row):
  score=0
  if pd.notna(row['PE_Ratio']) and row['PE_Ratio']<25:
    score+=1
    #Profitability of a company
  if pd.notna(row['ROE'])and row['ROE']>15:
    score+=1
  if pd.notna(row['Profit_Margin'])and row['Profit_Margin']>100:
    score+=1
  if(pd.notna(row['Revenue_Growth']))and row['Revenue_Growth']>10:
    score+=1
  if pd.notna(row['Returns_1Y']) and row['Returns_1Y'] > 0.15:
     score += 1
  #Judging the financial health of the company
  if pd.notna(row['Debt_to_Equity']) and row['Debt_to_Equity'] <100:
    score+=1
  if pd.notna(row['Current_Ratio']) and row['Current_Ratio'] > 1.5:
    score += 1
    #Growth of the company
  return score





In [ ]:
df1['Score'] = df1.apply(create_stock_rating, axis=1)

In [ ]:
df1

In [ ]:
def assign_hold_sell_buy(row):
  if row['Score']>=5:
    return "BUY"
  if row['Score']>=3 and row['Score']<5:
    return "HOLD"
  if row['Score']<3:
    return 'SELL'
  return None

In [ ]:
df1["Hold/Sell/Buy"]=df1.apply(assign_hold_sell_buy,axis=1)

In [ ]:
df1

Applying Machine Learning Algorithms
 (Import all the required Libraries)

In [ ]:
df2=df1# create a copy (Just in Case)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

In [ ]:
encoder=LabelEncoder()
df2['Hold/Sell/Buy']=encoder.fit_transform(df2['Hold/Sell/Buy'])

In [ ]:
df2.head()

1->sell
0->hold
2->buy

In [ ]:
features=['PE_Ratio','Price_to_Book','ROE','Profit_Margin','Debt_to_Equity','Revenue_Growth','Current_Ratio','Returns_1Y']
X=df2[features].fillna(df2[features].mean())# if there is missing value we will fill it the mean of the all the value of row instead of NaN

In [ ]:

y=df2['Hold/Sell/Buy']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.8,random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Scale training data
X_train_scaled = scaler.fit_transform(X_train)#when we scale the data we loose the column name as it gets converted it into numpy array to prevent that we convert that in dataframe with column name
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)

# Scale test data using the same scaler
X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_train.columns, index=X_test.index)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB

# Define base models
lr = LogisticRegression(max_iter=200, class_weight='balanced')
svc = SVC(kernel='rbf', probability=True, class_weight='balanced')
knn = KNeighborsClassifier(n_neighbors=5)
dt = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(n_estimators=300, random_state=42)
nb = GaussianNB()
# Voting ensemble
voting = VotingClassifier(
    estimators=[('lr', lr), ('svc', svc), ('rf', rf), ('knn', knn)],
    voting='soft'
)

# Dictionary of models
# Dictionary of models
models = {
    "Logistic Regression": lr,
    "SVM": svc,
    "KNN": knn,
    "Decision Tree": dt,
    "Random Forest": rf,
    "Naive Bayes": nb,
    "Voting Ensemble": voting,
    "XGBoost": XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric='mlogloss'
    )
}


create a dictionary of the machine learnign models

In [ ]:
result={}
for name,model in models.items():
  if name in("Logistic Regression","SVM","KNN"):
    model.fit(X_train,y_train)
    pred=model.predict(X_test_scaled)


  else:
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
  accuracy=accuracy_score(y_test,pred)
  result[name]=accuracy
  print(f"Model:{name}")
  print("Accuracy:", accuracy)
  print("\nClassification Report:")
  print(classification_report(y_test, pred))






In [ ]:
result

Choose the Best perforrming model and do their hyperparamete tuning


Hyperparemeter Tuning


In [ ]:
sorted_result=dict(sorted(result.items(),key=lambda item:item[1],reverse=True))
top_model_name=list(sorted_result.keys())[0]
print("The best performing model is",top_model_name,"with an acuuracy of",sorted_result[top_model_name])

To improve the accuracy and its performance we will do Hyperparameter tuning inorder to increase the performace
and accuracy


In [ ]:
from sklearn.model_selection import GridSearchCV


In [ ]:
param_grid = {
        'n_estimators': [100, 200, 500],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2'] # 'auto' is deprecated and removed
    }

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# Fit on training data
grid_search.fit(X_train, y_train)

In [ ]:
print("\nBest Hyperparameters for Random Forest:")
print(grid_search.best_params_)

In [ ]:
print("\nBest Hyperparameters for Random Forest:")
print(grid_search.best_params_)

Graphs


In [ ]:
graph_df=pd.DataFrame(list(result.items()),columns=["Model","Accuracy"])

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.bar(graph_df['Model'],graph_df['Accuracy'],data=graph_df,width=0.4,)

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(df1)